In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_slat import slat_align, slat_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

%load_ext autoreload
%autoreload 2

In [2]:
order = [
    155, 153,151,149,147,144,142,140, 138, 143,141,139, 137,135,133,130,131,
129,127,125,123,121,119,117,115,113,111,109,107,105,103, 101, 99, 97, 95, 93,
91,89,87,85,81,83,79,77,75,73,71,67,65,63,61,59,57,55,53,51,49,47,45,43,41,39,
37,33,31,29,27,25,28,30,32,34,36,38,40,42,44,48,50,52,54,56,58,60,62,
    64,66,68,72,74,76,78,80,82,84,88,90,92,94,96,98,100,102,104,106,108,
    110,112,114,116,118,120,122,124,136,134,132,128,126
]
data_folder = "/dm7100f/yifan/StereoSeq_macaque_processed"
files = [f'macaque_T{o}.h5ad' for o in order]

In [3]:
# data_folder = "./results/split_data/"
results_folder = "./results/SLAT/"
figures_folder = "./results/figures/SLAT"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [4]:
## Perform the Spateo alignment
from tqdm import tqdm
import time
sampling_num = 20000
spatial_key = 'r_spatial'
key_added = 'align_spatial'
for i in tqdm(range(len(files)-1)):
    slice1 = ad.read_h5ad(os.path.join(data_folder, files[i]))
    slice2 = ad.read_h5ad(os.path.join(data_folder, files[i+1]))
    slice1.obsm[spatial_key] = slice1.obsm[spatial_key].astype(np.float64)
    slice2.obsm[spatial_key] = slice2.obsm[spatial_key].astype(np.float64)
    slice1 = slice1[~np.isnan(slice1.obsm[spatial_key].sum(1))]
    slice2 = slice2[~np.isnan(slice2.obsm[spatial_key].sum(1))]
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    top_genes_num = 5000
    top_genes_1 = np.argsort(-slice1.X.sum(0)).A[0][:top_genes_num] if slice1.shape[1] > top_genes_num else np.arange(slice1.shape[1])
    top_genes_2 = np.argsort(-slice2.X.sum(0)).A[0][:top_genes_num] if slice2.shape[1] > top_genes_num else np.arange(slice2.shape[1])
    slice1 = slice1[:,top_genes_1]
    slice2 = slice2[:,top_genes_2]
    
    time_start = time.time()
    align_slices, pis = slat_align(
        models = [slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
    )
    # align_slices[1].uns['align_spatial_iter'] = morpho_model.iter_added
    time_end = time.time()
    matches = pis
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                    | 0/118 [00:00<?, ?it/s]

Calculating spatial neighbor graph ...
The graph contains 199095 edges, 17613 cells.
11.303866462272186 neighbors per cell on average.
Calculating spatial neighbor graph ...
The graph contains 227038 edges, 20000 cells.
11.3519 neighbors per cell on average.
Use DPCA feature to format graph


/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Mak

Warning! Dual PCA is using GPU, which may lead to OUT OF GPU MEMORY in big dataset!
Choose GPU:0 as device
Running
---------- epochs: 1 ----------
---------- epochs: 2 ----------
---------- epochs: 3 ----------
---------- epochs: 4 ----------
---------- epochs: 5 ----------
---------- epochs: 6 ----------
Training model time: 4.33


  1%|▏                         | 1/118 [00:35<1:09:08, 35.46s/it]

...

 99%|███████████████████████▊| 117/118 [1:12:31<00:38, 38.23s/it]

Calculating spatial neighbor graph ...
The graph contains 227490 edges, 20000 cells.
11.3745 neighbors per cell on average.
Calculating spatial neighbor graph ...
The graph contains 225734 edges, 20000 cells.
11.2867 neighbors per cell on average.
Use DPCA feature to format graph


/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Mak

Warning! Dual PCA is using GPU, which may lead to OUT OF GPU MEMORY in big dataset!
Choose GPU:0 as device
Running
---------- epochs: 1 ----------
---------- epochs: 2 ----------
---------- epochs: 3 ----------
---------- epochs: 4 ----------
---------- epochs: 5 ----------
---------- epochs: 6 ----------
Training model time: 2.68


100%|████████████████████████| 118/118 [1:13:03<00:00, 37.15s/it]


In [ ]:
## Perform the SLAT alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = slat_align(
        models = [slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
    )
    time_end = time.time()
    matches = pis
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)